In [1]:
import csv
import ast
import re
import numpy as np
import pandas as pd
import seaborn as sns

from Bio import SeqIO

from matplotlib import pyplot as plt

In [2]:
def cigarsfromsam(samfilepath):
    cigars = {}
    with open(samfilepath,"r") as samfile:
        for line in samfile:
            if line[0]=="@":
                next(samfile)
            else:
                splitline = line.split("\t")
                cigars[splitline[0]] = splitline[5]
    return cigars

def strsfromfasta(fastafilepath):
    queries = SeqIO.to_dict(SeqIO.parse(fastafilepath,"fasta"))
    queries = {key: str(val.seq) for key,val in queries.items()}
    return queries

def make_seg_dict(gfafile):
    segment_dict = {}
    with open(gfafile,"r") as infile:
        for line in infile:
            if line[0] == "S":
                splitline = line.split("\t")
                segment_dict[splitline[1]] = splitline[2][:-1]
    return segment_dict

def get_ref_intervals(gfafile):
    segment_dict = {}
    current_idx = 0
    with open(gfafile,"r") as infile:
        for line in infile:
            if line[0] == "S":
                splitline = line.split("\t")
                if "OFF" not in splitline[1]:
                    refstr = splitline[2][:-1]
                    strlen = len(refstr)
                    name = splitline[1]
                    if "ON" in name:
                        name=name[:-2]
                    segment_dict[name] = tuple((current_idx,current_idx+strlen))
                    current_idx += strlen
    return segment_dict
    
def align_read(querystr,cigarstr,pattern=re.compile("[0-9]{0,10}[MX=DI]")):
    result = pattern.finditer(cigarstr)
    cigar_seq = [(item.group(0)[-1],int(item.group(0)[:-1])) for item in result]
#     output_str = "".join(["-" for i in range(cigar[1])])
    output_str = ""
    current_idx = 0
    for item in cigar_seq:
        if any([item[0] == "M",item[0] == "=",item[0] == "X"]):
            added_str = querystr[current_idx:current_idx+item[1]]
            output_str += added_str
            current_idx += item[1]
        elif item[0]=="D":
            added_str = "".join(["-" for i in range(item[1])])
            output_str += added_str
        elif item[0]=="I":
            current_idx += item[1]
    return output_str

def align_read_sanger(querystr, refstr, cigarstr, startpos=1, pattern=re.compile("[0-9]{0,10}[MX=DI]")):
    start_pos = startpos - 1  ##comes as 1 indexed from minimap
    result = pattern.finditer(cigarstr)
    cigar_seq = [(item.group(0)[-1], int(item.group(0)[:-1])) for item in result]
    output_str = ""
    if start_pos > 0:
        output_str += "".join(["-" for i in range(start_pos)])
    current_idx = 0
    for item in cigar_seq:
        if any([item[0] == "M",item[0] == "=",item[0] == "X"]):
            added_str = querystr[current_idx : current_idx + item[1]]
            output_str += added_str
            current_idx += item[1]
        elif item[0] == "D":
            added_str = "".join(["-" for i in range(item[1])])
            output_str += added_str
        elif item[0] == "I":
            current_idx += item[1]
    remaining_len = len(refstr) - len(output_str)
    if remaining_len > 0:
        output_str += "".join(["-" for i in range(remaining_len)])
    return output_str

def splitstr(instr,ref_intervals):    
    strassign = {key:instr[val[0]:val[1]] for key,val in ref_intervals.items()}
    return strassign

def slow_hamming_distance(s1, s2):
    if len(s1) != len(s2):
        print(s1,s2)
        raise ValueError("Strand lengths are not equal!")
    term_list = []
    for ch1,ch2 in zip(s1,s2):
        if ch1 == "N" or ch2 == "N":
            term_list.append(False)
        else:
            term_list.append(ch1 != ch2)
    result = sum(term_list)
    return result

def get_dict_dist(dict1,dict2):
    hamming_dict = {key:slow_hamming_distance(dict1[key],dict2[key]) for key in dict1.keys()}
    return hamming_dict

In [44]:
# Imports
R9_data = pd.read_csv("/home/de64/group/de64/CRISPRi_Libraries/dev_notebooks/2024-11-23_Figure_Notebooks/Data/lDE11_Sequencing/lDE11_R9/output.tsv",delimiter="\t")
R10_data = pd.read_csv("/home/de64/group/de64/CRISPRi_Libraries/dev_notebooks/2024-11-23_Figure_Notebooks/Data/lDE11_Sequencing/lDE11_R10/output.tsv",delimiter="\t")
ref_intervals = get_ref_intervals("/home/de64/group/de64/CRISPRi_Libraries/dev_notebooks/2024-11-23_Figure_Notebooks/Data/lDE11_Sequencing/lDE11.gfa")

### Get Sequencing Dataframes

In [45]:
R9_barcodes = set(R9_data["barcode"].tolist())
R10_barcodes = set(R10_data["barcode"].tolist())

R9_only_barcodes = R9_barcodes-R10_barcodes
R10_only_barcodes = R10_barcodes-R9_barcodes
shared_barcodes = R10_barcodes&R9_barcodes
barcode_count_arr = np.array([len(R9_only_barcodes),len(R10_only_barcodes),len(shared_barcodes)])

R9_data = R9_data[R9_data["barcode"].isin(shared_barcodes)]
R10_data = R10_data[R10_data["barcode"].isin(shared_barcodes)]

In [46]:
aligned_cons = R9_data.apply(lambda x: align_read(x["consensus"],x["cigar"]), axis=1)
R9_data["aligned_cons"] = aligned_cons

aligned_cons = R10_data.apply(lambda x: align_read(x["consensus"],x["cigar"]), axis=1)
R10_data["aligned_cons"] = aligned_cons

split_ref = R9_data.apply(lambda x: splitstr(x["reference"],ref_intervals), axis=1)
split_align = R9_data.apply(lambda x: splitstr(x["aligned_cons"],ref_intervals), axis=1)
R9_data["split_ref"] = split_ref
R9_data["split_align"] = split_align

split_ref = R10_data.apply(lambda x: splitstr(x["reference"],ref_intervals), axis=1)
split_align = R10_data.apply(lambda x: splitstr(x["aligned_cons"],ref_intervals), axis=1)
R10_data["split_ref"] = split_ref
R10_data["split_align"] = split_align

R9_data["split_ref"] = R9_data["split_ref"].apply(lambda x: {key:val for key,val in x.items() if key=="GFP"}) ## This is a hack until I can repull the alignment data...then I'll add padding for the unaligned parts
R9_data["split_align"] = R9_data["split_align"].apply(lambda x: {key:val for key,val in x.items() if key=="GFP"}) ## This is a hack until I can repull the alignment data...

R10_data["split_ref"] = R10_data["split_ref"].apply(lambda x: {key:val for key,val in x.items() if key=="GFP"}) ## This is a hack until I can repull the alignment data...then I'll add padding for the unaligned parts
R10_data["split_align"] = R10_data["split_align"].apply(lambda x: {key:val for key,val in x.items() if key=="GFP"}) ## This is a hack until I can repull the alignment data...

hamm_ref = R9_data.apply(lambda x: get_dict_dist(x["split_align"],x["split_ref"]), axis=1)
R9_data["hamm_ref"] = hamm_ref
hamm_ref = R10_data.apply(lambda x: get_dict_dist(x["split_align"],x["split_ref"]), axis=1)
R10_data["hamm_ref"] = hamm_ref

dark_gfp = R9_data.apply(lambda x: slow_hamming_distance(x["split_align"]["GFP"][623:625],x["split_ref"]["GFP"][623:625]), axis=1)>0
R9_data["dark_gfp"] = dark_gfp
dark_gfp = R10_data.apply(lambda x: slow_hamming_distance(x["split_align"]["GFP"][623:625],x["split_ref"]["GFP"][623:625]), axis=1)>0
R10_data["dark_gfp"] = dark_gfp

R9_data["UMI"] = R9_data["split_align"].apply(lambda x: x["GFP"][925:940])
R10_data["UMI"] = R10_data["split_align"].apply(lambda x: x["GFP"][925:940])

R9_data.to_csv("/home/de64/group/de64/CRISPRi_Libraries/dev_notebooks/2024-11-23_Figure_Notebooks/Data/lDE11_Sequencing/lDE11_R9_df.tsv", sep="\t")
R10_data.to_csv("/home/de64/group/de64/CRISPRi_Libraries/dev_notebooks/2024-11-23_Figure_Notebooks/Data/lDE11_Sequencing/lDE11_R10_df.tsv", sep="\t")

TypeError: align_read() missing 1 required positional argument: 'cigarstr'